In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')

driver = webdriver.Chrome(chrome_options = options, executable_path = "/home/cosbi/financialSite/DB/crawler/twse/chromedriver")

/tmp/ipykernel_1356142/3466608402.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_options = options, executable_path = "/home/cosbi/financialSite/DB/crawler/twse/chromedriver")
/tmp/ipykernel_1356142/3466608402.py:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options = options, executable_path = "/home/cosbi/financialSite/DB/crawler/twse/chromedriver")


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 100
Current browser version is 102.0.5005.61 with binary path /usr/bin/google-chrome
Stacktrace:
#0 0x55ccf8c7dad3 <unknown>
#1 0x55ccf89dd568 <unknown>
#2 0x55ccf8a046b5 <unknown>
#3 0x55ccf89ffe54 <unknown>
#4 0x55ccf89fc23a <unknown>
#5 0x55ccf8a372ba <unknown>
#6 0x55ccf8a314e3 <unknown>
#7 0x55ccf8a06d1a <unknown>
#8 0x55ccf8a07e75 <unknown>
#9 0x55ccf8cabefd <unknown>
#10 0x55ccf8cc519b <unknown>
#11 0x55ccf8cadc65 <unknown>
#12 0x55ccf8cc5ec8 <unknown>
#13 0x55ccf8ca1360 <unknown>
#14 0x55ccf8ce1a68 <unknown>
#15 0x55ccf8ce1be8 <unknown>
#16 0x55ccf8cfb7fd <unknown>
#17 0x7fdee3d1a609 <unknown>
